<a href="https://colab.research.google.com/github/KidLiumingjie/Text_Classification/blob/main/Text_Classification(practice).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 取得model連結後直接到第二步

# 第一步:訓練模型

In [4]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [5]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 保存datasets to Google Drive
from datasets import load_dataset

dataset = load_dataset("fancyzhx/ag_news")
#dataset.save_to_disk('/content/drive/MyDrive/ag_news_dataset')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [7]:
# 加載模型和分詞器
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding

model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)
tokenizer = BertTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# 加載數據並拆分
train_test_split = dataset['train'].train_test_split(test_size=0.1)
train_data = train_test_split['train']
test_data = train_test_split['test']

In [9]:
# 數據預處理

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

encoded_train = train_data.map(preprocess_function, batched=True)
encoded_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/108000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [10]:
#評估指標
import numpy as np
from datasets import load_metric

accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}


<ipython-input-10-40772d52ba0c>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


The repository for f1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/f1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


In [11]:

# 設置訓練參數
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    lr_scheduler_type="linear" , # 學習率設置
    fp16=True,  # 混合精度啟用
)

# 數據整理器
data_collator = DataCollatorWithPadding(tokenizer)

# 創建 trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # 加入評估函數
)

# train model
trainer.train()



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.183000,0.178707,0.948250,0.948267
2,0.129600,0.177410,0.951167,0.951214


TrainOutput(global_step=13500, training_loss=0.18030121132179544, metrics={'train_runtime': 4528.2262, 'train_samples_per_second': 47.701, 'train_steps_per_second': 2.981, 'total_flos': 3.906653479052813e+16, 'train_loss': 0.18030121132179544, 'epoch': 2.0})

In [12]:
# 評估model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Evaluation Results: {'eval_loss': 0.17741043865680695, 'eval_accuracy': 0.9511666666666667, 'eval_f1': 0.951214212227894, 'eval_runtime': 52.0637, 'eval_samples_per_second': 230.487, 'eval_steps_per_second': 14.405, 'epoch': 2.0}


In [ ]:
# 保存模型到 Google Drive
model.save_pretrained('/content/drive/MyDrive/bert_ag_news_model')
tokenizer.save_pretrained('/content/drive/MyDrive/bert_ag_news_model')

# 第二步:使用模型

In [14]:
!pip install transformers datasets torch

In [15]:
# Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_from_disk
from transformers import BertForSequenceClassification, BertTokenizer


In [ ]:

# 加載數據集
dataset = load_from_disk('/content/drive/MyDrive/ag_news_dataset')
train_data = dataset['train']
test_data = dataset['test']

In [ ]:
# 加載模型
model_name_or_path = '/content/drive/MyDrive/bert_ag_news_model'
model = BertForSequenceClassification.from_pretrained(model_name_or_path)
tokenizer = BertTokenizer.from_pretrained(model_name_or_path)


In [19]:
# define labels
labels = ["World", "Sports", "Business", "Science/Technology"]

In [20]:
# 分類函數
def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax().item()
    return labels[predicted_class]

In [21]:
# 輸入文本分類
text = "The stock market saw a significant increase today as new technology stocks soared."
predicted_category = classify_text(text)
print(f"Predicted Category: {predicted_category}")

Predicted Category: Business
